# NNLM

NNLM是从语言角度出发（即计算概率角度），构建神经网络针对目标函数对模型进行优化，训练的起点是使用神经网络去搭建语言模型实现词的预测任务，模型的产品就是词向量。

进行神经网络模型的训练时，目标是进行词的概率预测，就是在词环境下，预测一下该是什么词，目标含糊入校通过对网络训练一定程度后，最后的模型参数就可以当成向量使用，

$$
L = \sum_{t}logP(w_t|w_{t-n+1},...w_{t-1})
$$

# 模型
NNLM的网络结构（四层神经网络）。主要参数有：
- 词向库大小(假定有8W个词)
- 转化的词向量大小（假定为400维长度）
- 输入层神经元数（即词的滑动窗口容量，假定滑动窗口大小为4）
- 隐藏神经元数（假定为100个）
- 输出层神经元个数（对应词容量，有8W个）
- 由输入层到投影层的矩阵C（一个大矩阵，大小为8W*300 是最后求解的目的，开始是初始化）
- 从投影层到隐层的权值矩阵H和偏置矩阵B
- 从隐层到输出层的权值矩阵U和偏置层矩阵D

<img src="picture/201_101.png">

对NNLM模型进行分析

- 每次从语料库中滑动4个数据，将其中前三个词转化为one-hot形式作为输入喂入网络。
- 从输入到映射做的事情是（ont-hot向量*矩阵C），这里词的one-hot编码会根据为1位置对对应C矩阵抽出对应位置的300维的词向量，将词向量作为投影输出。
- 上一步投影层会将词的one-hot表示成300维的稠密向量，从投影层到隐层是一种全连接方式，线的数量是3*100个，每个隐层由3条线连接，最后使用tanh函数结合H与B获取激活输出。
- 从隐层到输出层也是一直全连接的形式，连接数量为100*8W，使用softmax函数结合U与D获取最后的概率输出。
- 计算交叉熵损失函数，以梯度下降方式进行反向传播，在反向传播过程中参数矩阵C,H,B,U,D进行更新。
-

通过不断的喂入批次数据，对网络进行反向传播调参，最后训练处一个进行词预测任务的模型，并训练好的模型中的C矩阵里的每一列都作为对应one-hot编码中位置为1词的词向量（大小为1*300），这个词向量就是我们要转化的结果。


**优点**
- 使用NNLM模型生成的词向量是可以自定义维度的，维度并不会因为新词扩展而发生改变，而且这里生成的词向量能够很好的根据特征距离度量词与词之间的相似性。

**缺点**
- 计算复杂度过大，参数较多

In [4]:
from tensorflow.keras import layers
from tensorflow import keras

In [5]:
vocab_size = 5000
input_shape = (50, )

model = keras.Sequential()
model.add(layers.Embedding(input_dim=vocab_size, output_dim=30, input_shape=input_shape))
model.add(layers.Flatten()) # 摊平(concat)
model.add(layers.Dense(128, activation='tanh')) # Hidden layer
model.add(layers.Dense(vocab_size, activation='softmax'))  # Output layer
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 30)            150000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1500)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               192128    
_________________________________________________________________
dense_3 (Dense)              (None, 5000)              645000    
Total params: 987,128
Trainable params: 987,128
Non-trainable params: 0
_________________________________________________________________
